In [1]:
# ensure cwd is project root
import os
root = r"C:\Users\Dell\Documents\Customer-Churn-Prediction"
if os.getcwd() != root:
    os.chdir(root)
print("cwd:", os.getcwd())


cwd: C:\Users\Dell\Documents\Customer-Churn-Prediction


In [2]:
# dataset check
from pathlib import Path
p = Path("data")/"raw"
print("expected dataset path:", p.resolve())
print("exists?", p.exists())
if p.exists():
    files = list(p.iterdir())
    print("files in data/raw:")
    for f in files:
        print("-", repr(f.name), f.stat().st_size, "bytes")
else:
    print("data/raw not found. Create data/raw and place churn_modelling.csv inside.")

expected dataset path: C:\Users\Dell\Documents\Customer-Churn-Prediction\Data\raw
exists? True
files in data/raw:
- 'Churn_Modelling.csv' 684858 bytes


In [3]:
# quick load first rows & shape
import pandas as pd
from pathlib import Path

data_path = Path("data")/"raw"/"churn_modelling.csv"
if not data_path.exists():
    raise FileNotFoundError(f"{data_path} not found. Put dataset at this path.")
df = pd.read_csv(data_path)
print("rows, cols:", df.shape)
display(df.head(5))

rows, cols: (10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#  separate X, y and inspect numeric/categorical
if "Exited" not in df.columns:
    raise ValueError("CSV must contain 'Exited' target column.")

y = df["Exited"].astype(int)
X = df.drop(columns=["RowNumber","CustomerId","Surname","Exited"], errors="ignore")
num_cols = X.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Numeric cols:", num_cols)
print("Categorical cols:", cat_cols)
display(X.describe().T)

Numeric cols: ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
Categorical cols: ['Geography', 'Gender']


,count,mean,std,min,25%,50%,75%,max
CreditScore,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
Age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48


In [6]:
import sklearn; print(sklearn.__version__)

1.7.2


In [8]:
# construct transformers
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preproc = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])
print("Preprocessing pipeline ready.")

Preprocessing pipeline ready.
